In [23]:
# !pip install deeprobust
# !conda install pytorch torchvision torchaudio -c pytorch
import torch
# print(torch.__version__)
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-geometric
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis

import random



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_6973/273441208.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [24]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

import numpy
import torch
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


from random import sample
from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

import os
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_6973/4057615484.py:25: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


'/Users/prakashpal/Desktop/Sandip Sir/Entropy_Code'

In [25]:
dataset = os.path.join(os.getcwd(),'Cora')
dataset


'/Users/prakashpal/Desktop/Sandip Sir/Entropy_Code/Cora'

In [26]:
import os.path as osp
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

def get_planetoid_dataset(name, normalize_features=False, transform=None, split="public"):
    path = osp.join(osp.dirname(osp.realpath(os.getcwd())), '..', 'data', name)
    if split == 'complete':
        dataset = Planetoid(path, name)
        dataset[0].train_mask.fill_(False)
        dataset[0].train_mask[:dataset[0].num_nodes - 1000] = 1
        dataset[0].val_mask.fill_(False)
        dataset[0].val_mask[dataset[0].num_nodes - 1000:dataset[0].num_nodes - 500] = 1
        dataset[0].test_mask.fill_(False)
        dataset[0].test_mask[dataset[0].num_nodes - 500:] = 1
    else:
        dataset = Planetoid(path, name, split=split)
    if transform is not None and normalize_features:
        dataset.transform = T.Compose([T.NormalizeFeatures(), transform])
    elif normalize_features:
        dataset.transform = T.NormalizeFeatures()
    elif transform is not None:
        dataset.transform = transform
    return dataset


if __name__ == '__main__':
#     lst_names = ['Cora', 'CiteSeer', 'PubMed']
    lst_names = ['Cora']
    for name in lst_names:
        dataset = get_planetoid_dataset(name)
        print(f"dataset: {name}")
        print(f"num_nodes: {dataset[0]['x'].shape[0]}")
        print(f"num_edges: {dataset[0]['edge_index'].shape[1]}")
        print(f"num_classes: {dataset.num_classes}")
        print(f"num_features: {dataset.num_node_features}")

dataset: Cora
num_nodes: 2708
num_edges: 10556
num_classes: 7
num_features: 1433


In [27]:
# from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj

# # dataset = NELL(root='/nell')
k_ = dataset.num_classes

# dataset= Planetoid(root=dataset, name='Cora')
print(dataset[0])
adj = to_dense_adj(dataset[0].edge_index)
adj = adj[0]
labels = dataset[0].y
labels = labels.numpy()

X = dataset[0].x
X = X.to_dense()
N = X.shape[0]
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))

print(X.shape, adj.shape)

nn = int(1*N)
X = X[:nn,:]
adj = adj[:nn,:nn]
A = adj[:nn,:nn]
AT= torch.transpose(A,0,1)
labels = labels[:nn]
print(X.shape,adj.shape)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
torch.Size([2708, 1433]) torch.Size([2708, 2708])
torch.Size([2708, 1433]) torch.Size([2708, 2708])


In [28]:
X.shape

torch.Size([2708, 1433])

In [29]:
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)

torch.Size([2708, 2708])


In [30]:
import torch
import torch.nn.functional as F

##### Formula  entropy = -torch.sum(normalized_eigenvalues * torch.log2(normalized_eigenvalues))

A = theta
print(type(A))
# eigenvalues = torch.eig(A, eigenvectors=False)[0][:, 0]
eigenvalues = torch.linalg.eigvals(A)
nonzero_eigenvalues = eigenvalues[eigenvalues != 0]
sum_nonzero_eigenvalues = nonzero_eigenvalues.sum()
normalized_eigenvalues = nonzero_eigenvalues / sum_nonzero_eigenvalues
epsilon = 1e-8
p = torch.log(normalized_eigenvalues + epsilon)
lm = normalized_eigenvalues * p
entropy = -torch.sum(lm)
entropy = entropy.item()
print("Entropy:", entropy)

<class 'torch.Tensor'>
Entropy: (7.423994064331055-0j)


In [31]:
(theta@A).type()

'torch.FloatTensor'

In [32]:
## Delete later
theta.device

device(type='cpu')

In [33]:

features = torch.Tensor(X)
NO_OF_NODES = X.shape[0]
print(NO_OF_CLASSES,NO_OF_NODES)

7 2708


In [34]:
## Delete later
features.device

device(type='cpu')

In [35]:
def convertScipyToTensor(coo):
  try:
    coo = coo.tocoo()
  except:
    coo = coo
  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [36]:
from scipy.sparse import random
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix

p = X.shape[0]
k = int(p*0.1)
n = X.shape[1]
lr = 1e-5
thresh = 1e-10

from scipy.sparse import random
from scipy.stats import rv_continuous
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
temp = CustomDistribution(seed=1)
temp2 = temp()  # get a frozen version of the distribution
C = random(p, k, density=0.25, random_state=1, data_rvs=temp2.rvs)




In [37]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    
####### NO output layer is written


In [38]:
import torch
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
##### Formula  entropy = -torch.sum(normalized_eigenvalues * torch.log2(normalized_eigenvalues))


# def entropy_(Coarse_Mat):
#     A = Coarse_Mat
#     A = torch.tensor(A)
# #     eigenvalues = torch.eig(A, eigenvectors=False)[0][:, 0]
#     eigenvalues = torch.linalg.eigvals(A)
#     nonzero_eigenvalues = eigenvalues[eigenvalues != 0]
#     sum_nonzero_eigenvalues = nonzero_eigenvalues.sum()
#     normalized_eigenvalues = nonzero_eigenvalues / sum_nonzero_eigenvalues
#     epsilon = 1e-8
#     p = torch.log(normalized_eigenvalues + epsilon)
#     lm = normalized_eigenvalues * p
#     entropy = -torch.sum(lm)
#     entropy = entropy.item()
#     print("Entropy after coarsen:", entropy)
#     return entropy
import numpy as np
import math

def entropy_(en_mat):
  EV, _ = np.linalg.eig(en_mat)
  my_list = [x for x in EV.tolist() if x]
  leng = len(my_list)
  Sum = np.sum(np.real(my_list))
  entropy = 0

  for i in range(leng):
      ratio = np.real(my_list[i]) / Sum
      if ratio > 0:
        entropy += -(ratio * math.log2(ratio))
  print("Entropy after coarsen:", entropy)

In [39]:
def get_accu(C_0,L,X_t_0):
    global labels, NO_OF_CLASSES,k
    t=[]
    for i in [1]: 
        C_0_new=np.zeros(C_0.shape)
        for i in range(C_0.shape[0]):
            C_0_new[i][np.argmax(C_0[i])]=1
        # print(C_0_new)
        # C_0_new=C_0
        from scipy import sparse
        #Lc=C_0.T@L@C_0
        
        Lc=C_0_new.T@L@C_0_new
        Lc_E = Lc
        Entropy_after_coarsening = entropy_(Lc_E)
        # print("L:", Lc.shape)
        # Lc=L_new
        #print(Lc)
        Wc=(-1*Lc)*(1-np.eye(Lc.shape[0]))
        # print("W:", Wc.shape)
        Wc[Wc<0.1]=0
        Wc=sparse.csr_matrix(Wc)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen2 = torch.stack([row, col], dim=0)
        #print("edgecoarsen:", edge_index_coarsen2.shape)
        edge_weight = torch.from_numpy(Wc.data)
        #print("edgeweight:", edge_weight.shape)
        def one_hot(x, class_count):
            return torch.eye(class_count)[x, :]

        device = torch.device('cpu')
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        labels=labels
        Y = labels
        #print("Y:", Y.shape)
        Y = one_hot(Y,NO_OF_CLASSES)
        # NO_OF_CLASSES=Y.shape[1]
        P=np.linalg.pinv(C_0_new)
        labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
        #print("Lables:", labels_coarse.shape)

        #torch.Tensor(C2)@X
        Wc=Wc.toarray()
        #Wc[Wc<0.01]=0
        C2=np.linalg.pinv(C_0_new)
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model=Net().to(device)
        lr=0.01
        decay=0.0001
        features_= features.cpu().detach().numpy()
        try:
          X=np.array(features_.todense())
        except:
          X = np.array(features_)
        #print("X:",X.shape)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
        # criterion=torch.nn.CrossEntropyLoss()
        x=sample(range(0, int(k)), k)
      
        from datetime import datetime
        Xt=P@X
        # Xt=X_t_0
        def train():
            model.train()
            optimizer.zero_grad()
            out = model(torch.Tensor(Xt).to(device),edge_index_coarsen2)
            loss = F.nll_loss(out[x], labels_coarse[x])
            loss.backward()
            optimizer.step()
            return loss
        now1 = datetime.now()
        losses=[]
        for epoch in range(100):
            loss=train()
            losses.append(loss)
            if(epoch%100==0):
                print(f'Epoch: {epoch:03d},loss: {loss:.4f}')
        now2 = datetime.now()        
        pred=model(torch.Tensor(Xt).to(device),edge_index_coarsen2).argmax(dim=1)        
        def train_accuracy():
            model.eval()
            correct = (pred[x] == labels_coarse[x]).sum()
            acc = int(correct) /len(x)
            return acc
    
        t+=[(now2-now1).total_seconds()]

        zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
        adj_ = adj.cpu().detach().numpy()
        Wc=sparse.csr_matrix(adj_)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen = torch.stack([row, col], dim=0)
        edge_weight = torch.from_numpy(Wc.data)
        pred=model(torch.Tensor(X),edge_index_coarsen).argmax(dim=1)
        pred=np.array(pred)
        correct =(pred[zz]==labels[zz]).sum()
        acc = int(correct) /NO_OF_NODES
        return acc



In [40]:
def experiment_structure(alpha_param,lambda_param,beta_param,gamma_param,C,theta,X,A):
      p = X.shape[0]
      k = int(p*0.1)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      
      try:
        C = convertScipyToTensor(C)
        C = C.to_dense()
      except:
        C=C
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X
      if(torch.cuda.is_available()):
        print("GPU is available")
        C = C
        theta = theta
        X = X
        ones = ones
      
        
      def bracket_term2fun(C,CT,theta):
          # U  = update_U(C,theta).double()
          U = torch.stack(update_U(C, theta)).double()
          UT= torch.transpose(U,0,1)
          Lw = (CT @theta @C).double()
          lb= 1e-5
          ub = 1e+4
          beta = 0.5 
          lambda_ =  laplacian_lambda_update(lb, ub, beta, U, Lw, k_,C)   
          lambda_matrix =  torch.diag(lambda_,0)
#           print("U size",U.size())
          return U@lambda_matrix@UT
        
      def update_U(C,theta):
            
        CT= torch.transpose(C,0,1)
        product = CT @ theta @ C
        matrix = torch.tensor(product)  
        eigenvalues, eigenvectors = torch.linalg.eig(product)

        # select non-zero eigenvalues and eigenvectors
        non_zero_eigenvalues = []
        non_zero_eigenvectors = []
        for i in range(matrix.shape[0]):
            if matrix[i, i] != 0:
                non_zero_eigenvalues.append(eigenvalues[i])
                non_zero_eigenvectors.append(eigenvectors[:, i])
        U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
        return U    
 

      def update_C(C):
          CT = torch.transpose(C,0,1)
          C.size()
          t1 = alpha_param*(C@ones)
          bracket_term1 = (CT@theta@C)
          bracket_term2 = bracket_term2fun(C,CT,theta) 
          bracket_term = bracket_term1 - bracket_term2   # bracket term (CT*theta*C - U*lambda*UT)
          t22 = -2*(theta@C) 
#           print(t22.type())
          t3 = bracket_term1
          t7 = bracket_term2
          t6 = (CT@A@C)
          t5 = 2* beta_param*(A@C)
          t5 = t5.float()
          t4 = (1.0/k)
          t44 = t4*((torch.ones(k,k)).double())
#           print(t3.device)
#           print(t44.device)
          t8 = (t3 + t44)
          t9 = torch.pinverse(t8)                  #change it
          t9 = t9.float()
#           print(t9.type())
#           print(t9)
          t10 = (t22@t9)
          t11 = (t6 - t7)
          t11 = t11.float()
          t12 = (t5@t11)
          t13 = (t1 + t10 +t12)
        
          #t2 = beta_param*(theta@C@bracket_term.float())
          grad_fc= t13
          C_new=C-gamma_param*grad_fc
          C_new[C_new<thresh] = thresh
          for i in range(len(C_new)):
              C_new[i] = C_new[i]/torch.linalg.norm(C_new[i],1)
          return C_new        
            

        
        
        


      #We set c1 = 10−5 and c2 = 10^4 We observed that the experimental performances of the algorithms 
       #are not sensitive to different values of c1 and c2 as long as they are reasonably small and large,respectively
      # K is the number of smallest eigenvalues of the Laplacian matrix that are being ignored while updating the eigenvalues.
      def laplacian_lambda_update(lb, ub, beta, U, Lw, k, C):
        q = Lw.size(1) - k

        UT= torch.transpose(U,0,1)
        UT = UT.type(torch.float64)

        
        CT= torch.transpose(C,0,1)
        CT = CT.type(torch.float64)

        
        AC=(A@C).double()

        
        Af=(CT@AC).double()

        Af.device
        U.device
        dd = U@Af@UT
        
        product = dd
        matrix = torch.tensor(product)            

        non_zero_diag_elements = []
        for i in range(matrix.shape[0]):
            if matrix[i, i] != 0:
                non_zero_diag_elements.append(matrix[i, i])
            if len(non_zero_diag_elements) == len(matrix):
                break

        k = len(non_zero_diag_elements)
        d = torch.diag(torch.tensor(non_zero_diag_elements))

       
        lambda_ = 0.5 * (d + torch.sqrt(d.pow(2) + 4 / beta))
#         print(lambda_)
        lambda_,indices = torch.sort(lambda_, dim=- 1, descending=True)
        eps = 1
        condition = torch.stack([(lambda_[q] - ub) <= eps,
                         (lambda_[0] - lb) >= -eps]).all(dim=0)

#                                   (lambda_[1:(q)] - lambda_[0:(q-1)]) >= -eps])
        if condition.all():
            return lambda_
        else:
            greater_ub = lambda_ > ub
            lesser_lb = lambda_ < lb
            lambda_[greater_ub] = ub
            lambda_[lesser_lb] = lb
            condition = torch.stack([(lambda_[q] - ub) <= eps,
                         (lambda_[0] - lb) >= -eps]).all(dim=0)

#                                   (lambda_[1:q] - lambda_[0:(q-1)]) >= -eps])
            if condition.all():
                return lambda_
            else:
#                 print(lambda_)
                raise ValueError("eigenvalues are not in increasing order, consider increasing the value of beta")
            

      for i in tqdm(range(10)): #update C only 21
         C = update_C(C)
            
      return C

In [41]:
import seaborn as sns
import matplotlib.pylab as plt
        # sns.heatmap(C_0.T@C_0)
import time   
        
highest_accuracy=0
lambda_param = 0.001
#0.0001,0.0001,10,0.0001
for alpha_param in [1]:
  for beta_param in [10]:
      for gamma_param in [0.0001]:
            
        av = []
        for _ in range(20):
            avg_accuracy_all=[]
            X=X
            for _ in range(1):
              C = random(p, k, density=0.15, random_state=1, data_rvs=temp2.rvs)


              a = time.time()
              C_0 = experiment_structure(alpha_param,lambda_param,beta_param,gamma_param,C,theta,X,A)
              b = time.time()

              L = theta

              pseudo_C = torch.linalg.pinv(C_0)
              X_t_0 = pseudo_C@X
              C_test = C_0.cpu().detach().numpy()
              X_t_test = X_t_0.cpu().detach().numpy()
              L_test = L.cpu().detach().numpy() 
              c = time.time()
              acc = get_accu(C_test,L_test,X_t_test)
              d = time.time()
              # print("Time taken:", b-a+d-c)
              av.append(acc)
              if highest_accuracy<acc:
                highest_accuracy=acc
                print("Accuracy = " + str(acc) + " " + str(alpha_param)+" " + str(beta_param)+" "+str(gamma_param))
        print("Average accuracy = " + str(np.mean(av)*100)  + " +/- " + str(np.std(av)*100))
        print("Params =  " + str(alpha_param)+" " + str(beta_param)+" "+str(gamma_param))

  0%|                                                    | 0/10 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_6973/769224947.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_6973/769224947.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_6973/769224947.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.te

Entropy after coarsen: 6.079958179070125
Epoch: 000,loss: 1.9531
Accuracy = 0.7389217134416544 1 10 0.0001


100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  4.15it/s]


Entropy after coarsen: 6.342948931536872
Epoch: 000,loss: 1.9500


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.48it/s]


...

Entropy after coarsen: 6.234477336275573
Epoch: 000,loss: 1.9558


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.25it/s]


Entropy after coarsen: 6.070566189473185
Epoch: 000,loss: 1.9296


100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  4.62it/s]


Entropy after coarsen: 5.866591348797534
Epoch: 000,loss: 1.9311
Accuracy = 0.7540620384047267 1 10 0.0001
Average accuracy = 71.9940915805022 +/- 1.8227961923908336
Params =  1 10 0.0001


In [42]:
highest_accuracy*100

75.40620384047267